# Load dataset

In [1]:
import tools

In [2]:
edges, nodes, poss_set, neg_set = tools.sample_structural(
    '../datasets/directed/ego-twitter/train.in', N=10, directed=True)

# Make bipartite network

In [3]:
from graph_tool.all import *
g = Graph(directed=False)
g.add_vertex(2*max(nodes) + 2)

<generator object <genexpr> at 0x7f222cf45280>

In [4]:
groups = g.new_vertex_property("int")
for u in g.vertices():
    groups[u] = int(u) % 2
    
for edge in edges:
    if edge not in poss_set:
        u, w = map(int, edge.split())
        g.add_edge(g.vertex(2*u), g.vertex(2*w + 1))

In [ ]:
pos = sfdp_layout(g, max_iter=4)

# Scores

In [ ]:
from sklearn.metrics import roc_auc_score

In [ ]:
features = tools.TopologicalFeatures(g, pos, directed=True)
X, Y = tools.make_dataset(poss_set, neg_set, 
                    [features.dist])
print('di-SFDP:', roc_auc_score(Y, X))

In [ ]:
from sklearn.decomposition import NMF
model = NMF(n_components=10, init='random', random_state=0)

matrix = tools.make_sparse_matrix(train_set, nodes, poss_set, directed=True)

features = tools.MFFeatures(model, matrix)
X, Y = tools.make_dataset(poss_set, neg_set, 
                    [features.score])
print("NMF:", roc_auc_score(Y, X))

In [ ]:
from scipy.sparse import linalg
import numpy

matrix = tools.make_sparse_matrix(edges, nodes, poss_set, directed=True)
U, s, Vh = linalg.svds(matrix.asfptype(), k=30)

def score(u, w):
    return numpy.dot(U[u] * s, Vh.T[w])

X, Y = tools.make_dataset(poss_set, neg_set, 
                    [score])
print("svds:", roc_auc_score(Y, X))

# Cross-validation

In [ ]:
import cross_validation

cross_validation.cross_validation(
    'data/directed/ego-twitter/train.in', N=30, k=10, directed=True, max_iter=25)

NMF-10 : 0.681783 (+/- 0.005706)
svds-30 : 0.669102 (+/- 0.007080)
PA : 0.500746 (+/- 0.004502)
CN : 0.510429 (+/- 0.001466)
Adamic-Adar : 0.510434 (+/- 0.001465)
sfdp-directed : 0.758950 (+/- 0.039955)
NMF-30 : 0.677939 (+/- 0.006048)
svds-10 : 0.678128 (+/- 0.008806)
sfdp-default : 0.717397 (+/- 0.006382)
